In [80]:
#Imports
import pandas as pd
from pathlib import Path
from pandas.tseries.offsets import DateOffset
from sklearn.model_selection import train_test_split
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR



In [91]:
#Dataframe of historical NVDA stock created
tradingnvdi_df = pd.read_csv(
    Path("NVDADATA_DAILY.csv"), 
        index_col="Date", 
        infer_datetime_format=True, 
        parse_dates=True
)

# Review the DataFrame
trading_df.head()

C:\Users\equit\AppData\Local\Temp\ipykernel_19624\2155288785.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  tradingnvdi_df = pd.read_csv(


,Close,Close
Date,,
2015-01-02 16:00:00,5.03,34.73
2015-01-05 16:00:00,4.95,33.67
2015-01-06 16:00:00,4.80,32.76
2015-01-07 16:00:00,4.78,33.00
2015-01-08 16:00:00,4.97,34.73


In [92]:
#Dataframe of historical NVDA stock created
tradingsmci_df = pd.read_csv(
    Path("SMCIDATA.csv"), 
        index_col="Date", 
        infer_datetime_format=True, 
        parse_dates=True
)

# Review the DataFrame
trading_df.head()

C:\Users\equit\AppData\Local\Temp\ipykernel_19624\1562615122.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  tradingsmci_df = pd.read_csv(


,Close,Close
Date,,
2015-01-02 16:00:00,5.03,34.73
2015-01-05 16:00:00,4.95,33.67
2015-01-06 16:00:00,4.80,32.76
2015-01-07 16:00:00,4.78,33.00
2015-01-08 16:00:00,4.97,34.73


In [96]:
columns = ["NVDI Close", "SMCI Close"]
trading_df.columns = columns
trading_df.head()

,NVDI Close,SMCI Close
Date,,
2015-01-02 16:00:00,5.03,34.73
2015-01-05 16:00:00,4.95,33.67
2015-01-06 16:00:00,4.80,32.76
2015-01-07 16:00:00,4.78,33.00
2015-01-08 16:00:00,4.97,34.73


In [106]:
# Calculate the daily returns using the closing prices and the pct_change function
trading_df["actual_returns"] = trading_df["NVDI Close"].pct_change()


# Drop all NaN values from the DataFrame
trading_df = trading_df.dropna()

# Review the DataFrame
display(trading_df.head())
display(trading_df.tail())

KeyError: 'NVDI Close'

In [104]:
# Define a window size of 4
short_window = 2

# Create a simple moving average (SMA) using the short_window and assign this to a new columns called sma_fast
trading_df['sma_fast'] = trading_df['Close'].rolling(window=short_window).mean()

ValueError: Cannot set a DataFrame with multiple columns to the single column sma_fast

In [102]:
#Combine dataframes
trading_df = pd.concat([tradingnvdi_df, tradingsmci_df], axis='columns', join='inner')
trading_df.head()

,Close,Close
Date,,
2015-01-02 16:00:00,5.03,34.73
2015-01-05 16:00:00,4.95,33.67
2015-01-06 16:00:00,4.80,32.76
2015-01-07 16:00:00,4.78,33.00
2015-01-08 16:00:00,4.97,34.73


In [99]:
# Define a window size of 100
long_window = 1000

# Create a simple moving average (SMA) using the long_window and assign this to a new columns called sma_slow
trading_df['sma_slow'] = trading_df['Close'].rolling(window=long_window).mean()

KeyError: 'Close'

In [64]:
# Drop the NaNs using dropna()
trading_df = trading_df.dropna()

In [65]:
# Assign a copy of the sma_fast and sma_slow columns to a new DataFrame called X
X = trading_df[['sma_fast', 'sma_slow']].copy()

# Display sample data
display(X.head())
display(X.tail())



,sma_fast,sma_slow
Date,,
2018-12-21 16:00:00,33.085,28.69367
2018-12-24 13:00:00,32.080,28.72049
2018-12-26 16:00:00,32.525,28.74897
2018-12-27 16:00:00,33.035,28.77698
2018-12-28 16:00:00,33.100,28.80542


,sma_fast,sma_slow
Date,,
2023-12-22 16:00:00,489.100,210.15408
2023-12-26 16:00:00,490.545,210.58760
2023-12-27 16:00:00,493.480,211.02179
2023-12-28 16:00:00,494.695,211.45691
2023-12-29 16:00:00,495.220,211.89137


In [66]:
# Create a new column in the trading_df called signal setting its value to zero.
trading_df['signal'] = 0.0

In [68]:
# Create the signal to buy
trading_df.loc[(trading_df['actual_returns'] >= 0), 'signal'] = 1

In [ ]:
# Create the signal to sell
trading_df.loc[(trading_df['actual_returns'] < 0), 'signal'] = -1

In [69]:
# Copy the new signal column to a new Series called y.
y = trading_df['signal'].copy()

In [71]:
## Split the Data Into Training and Testing Datasets


In [72]:
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2018-12-21 16:00:00


In [73]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)

# Display the training end date
print(training_end)

2019-03-21 16:00:00


In [ ]:
### Create the Training Datasets

In [74]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Display sample data
X_train.head()

,sma_fast,sma_slow
Date,,
2018-12-21 16:00:00,33.085,28.69367
2018-12-24 13:00:00,32.080,28.72049
2018-12-26 16:00:00,32.525,28.74897
2018-12-27 16:00:00,33.035,28.77698
2018-12-28 16:00:00,33.100,28.80542


In [ ]:
## Standardize the Data


In [83]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
## Indicators and Signals Generated 

In [ ]:
# Load your data into a DataFrame
# Replace the following lines with your own data loading code
# Example: data = pd.read_csv('your_data.csv')
data = pd.DataFrame({
    'security1': [10, 12, 15, 14, 16],
    'security2': [20, 22, 25, 24, 26],
    # Add more securities if needed
})

# Fit VAR model
model = VAR(data)

# Choose lag order using VAR order selection criteria (e.g., AIC)
# You may need to adjust the maximum lag order (maxlags) based on your data
# You can also use other information criteria such as BIC or HQIC
lag_order = model.select_order(maxlags=5)

# Fit VAR model with selected lag order
var_model = model.fit(maxlags=lag_order.aic)

# Print summary of VAR model
print(var_model.summary())

# Forecast future values
# Replace 'steps' with the number of steps you want to forecast
# Example: forecast = var_model.forecast(data.values[-lag_order:,:], steps=5)
forecast = var_model.forecast(data.values[-lag_order:,:], steps=5)
print("Forecasted values:")
print(forecast)